In [ ]:
import os
import cv2
import numpy as np




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def create_dir(path):
    """Create a directory"""
    if not os.path.exists(path):
        os.makedirs(path)

## SOBEL + CROP

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


image = cv2.imread(image_path)

grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Initialize the threshold value
threshold_value = 30

while True:

        sobel_x = cv2.Sobel(grayscale_image, cv2.CV_64F, 1, 0, ksize=3)
        sobel_y = cv2.Sobel(grayscale_image, cv2.CV_64F, 0, 1, ksize=3)
        edge_image = np.sqrt(sobel_x**2 + sobel_y**2)


        edge_mask = (edge_image > threshold_value).astype(np.uint8)

        # Find contours
        contours, _ = cv2.findContours(edge_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            break


        threshold_value -= 5

# Find the bounding box of the largest contour
largest_contour = max(contours, key=cv2.contourArea)
x, y, w, h = cv2.boundingRect(largest_contour)


mask = np.zeros_like(image)

# Draw the largest contour on the mask
cv2.drawContours(mask, [largest_contour], -1, (0, 255, 0), thickness=cv2.FILLED)


# Crop the original image using the bounding box
cropped_image = image[y:y+h, x:x+w]

print(image_path + " done")
plt.imshow(cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()



## SOBEL + ELLIPSE + CROP


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
def CropEllipse(image_path):
  # Load the image
  #image_path = r"C:\Users\miste\Desktop\Projet Aziza Othmana\DATASETS\BLASTOCYST_ main\blastocyst_main\Images\0001_01.png"
  image = cv2.imread(image_path)


  grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  sobel_x = cv2.Sobel(grayscale_image, cv2.CV_64F, 2, 1, ksize=5)
  sobel_y = cv2.Sobel(grayscale_image, cv2.CV_64F, 1, 2, ksize=5)
  edge_image = np.sqrt(sobel_x ** 2 + sobel_y ** 2)


  threshold_value = 30
  edge_mask = (edge_image > threshold_value).astype(np.uint8)


  contours, _ = cv2.findContours(edge_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  largest_contour = max(contours, key=cv2.contourArea)
  x1, y1, w1, h1 = cv2.boundingRect(largest_contour)


  mask = np.zeros_like(image)

  cv2.drawContours(mask, [largest_contour], -1, (0, 255, 0), thickness=cv2.FILLED)

  plt.imshow(cv2.cvtColor(mask, cv2.COLOR_BGR2RGB))

  # Parameters for ellipse creation
  p = x1 + w1 // 2  # X-coordinate of ellipse center
  k = y1 + h1 // 2  # Y-coordinate of ellipse center
  alpha =0          # Angle in degrees
  a = w1 // 2       # Semi-major axis length
  b = h1 // 2       # Semi-minor axis length


  segmentation_mask = np.zeros(image.shape[:2], dtype=np.uint8)


  for y in range(image.shape[0]):
      for x in range(image.shape[1]):
          # The ellipse equation
          term1 = ((x - p) * np.cos(np.radians(alpha)) + (y - k) * np.sin(np.radians(alpha))) ** 2
          term2 = a ** 2
          term3 = ((x - p) * np.sin(np.radians(alpha)) - (y - k) * np.cos(np.radians(alpha))) ** 2
          term4 = b ** 2
          value = term1 / term2 + term3 / term4

          # Check if the pixel falls within the ellipse
          if value <= 1:
              # Set the pixel to white
              segmentation_mask[y, x] = 255


  result_image = cv2.bitwise_and(image, image, mask=segmentation_mask)


  cropped_image = result_image[y1:y1+h1
                              , x1:x1+w1]


  resized_cropped_image = cv2.resize(cropped_image, (299, 299), interpolation=cv2.INTER_NEAREST)
  return cropped_image


In [ ]:
import os
from google.colab.patches import cv2_imshow
sav_path = "/content/drive/MyDrive/Cropped_images_AZIZAOTHMANA_Ellipse"
base_dir="/content/drive/MyDrive/DatasetAzizaOthmana/Images"
img="MTL-0130-0FD2-AB62-[47]-P03-FP3.jpg"
#for img in os.listdir(base_dir):
image_path = os.path.join(base_dir, img)
cropped_image=CropEllipse(image_path)
print(img + " done")
cv2_imshow(cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
    #cv2.imwrite(os.path.join(sav_path, img), cropped_image)



# Close any open plot window (if you're using plt for visualization)
plt.close()


DELETE FILES FROM DIRECTORY

In [ ]:
import os

folder_path = "/content/drive/MyDrive/Images-azizaothmmana/Cropped_images"

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Loop through the list of files and delete each one
for file in files:
    file_path = os.path.join(folder_path, file)
    try:
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        else:
            print(f"Not a file: {file_path}")
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

print("All files in the folder have been deleted.")


#  CANNY

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load the image
#image_path = "" #add the path
def Canny(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (3, 3), 0)
    edges = cv2.Canny(blurred_image, threshold1=50, threshold2=20)
    kernel = np.ones((3, 3), np.uint8)
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)
    contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    mask = np.zeros_like(image)
    cv2.drawContours(mask, [largest_contour], -1, (0, 255, 0), thickness=cv2.FILLED)
    cropped_image = image[y:y+h, x:x+w]
    enhanced_image = cv2.resize(crop_image, (299, 299))
    #plt.imshow(cv2.cvtColor(dilated_edges, cv2.COLOR_BGR2RGB))
    return cropped_image



In [ ]:
from google.colab.patches import cv2_imshow
import os
#There will be no output due to data confidentiality
sav_path = "/content/drive/MyDrive/Cropped_Image_Canny"
base_dir="/content/drive/MyDrive/DatasetAzizaOthmana/Images"
for img in os.listdir(base_dir):
    image_path = os.path.join(base_dir, img)
    cropped_image=Canny(image_path)
    print(img + " done")
    cv2_imshow(cropped_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite(os.path.join(sav_path, img), cropped_image)







##  Contrast Enhancement

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

# Load the image
image_path ="" #add the path
image = cv2.imread(image_path)

# Apply contrast enhancement using CLAHE (Contrast Limited Adaptive Histogram Equalization)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(1, 1))
enhanced_image = np.zeros_like(image)

for channel in range(3):  # Assuming a BGR color image
    enhanced_image[:, :, channel] = clahe.apply(image[:, :, channel])

cv2.imwrite('enhanced_image.jpg', enhanced_image)


cv2_imshow(image)
cv2_imshow(enhanced_image)
cv2.waitKey(0)
cv2.destroyAllWindows()



